In [1]:
import sys
!{sys.executable} -m pip install --user torch
import torch
print(torch.__version__)
!{sys.executable} -m pip install --user torchvision
import torchvision
print(torchvision.__version__)


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
2.6.0+cu124

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
0.21.0+cu124


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm  # For progress bars
import wandb
import json

################################################################################
# Model Definition (Simple Example - You need to complete)
# For Part 1, you need to manually define a network.
# For Part 2 you have the option of using a predefined network and
# for Part 3 you have the option of using a predefined, pretrained network to
# finetune.
################################################################################
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # TODO - define the layers of the network you will use
        # Define the first convolutional layer
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
        # Define the second convolutional layer
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=2)
        # Define the third convolutional layer
        self.conv3 = nn.Conv2d(32, 50, kernel_size=5, padding=2)
        # Define the max pooling layer with a 2x2 window
        self.pool = nn.MaxPool2d(2, 2)
        # Define a dropout layer with dropout probability of 0.5 to prevent overfitting
        self.dropout = nn.Dropout(0.5)
        # Define the first fully connected layer
        self.fc1 = nn.Linear(800, 256)
        # Define the second fully connected layer
        self.fc2 = nn.Linear(256, 100)

    
    def forward(self, x):
        # TODO - define the forward pass of the network you will use
        # Apply conv1
        x = self.pool(F.relu(self.conv1(x)))
        # Apply conv2
        x = self.pool(F.relu(self.conv2(x)))
        # Apply conv3
        x = self.pool(F.relu(self.conv3(x)))
        # Flatten the tensor for the fully connected layers
        x = x.view(-1, 800)
        # Apply fc1 with ReLU activation and then dropout
        x = self.dropout(F.relu(self.fc1(x)))
        # Apply the final fully connected layer
        x = self.fc2(x)
        return x


################################################################################
# Define a one epoch training function
################################################################################
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    """Train one epoch, e.g. all batches of one epoch."""
    device = CONFIG["device"]
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    # put the trainloader iterator in a tqdm so it can printprogress
    progress_bar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{CONFIG['epochs']} [Train]", leave=False)

    # iterate through all batches of one epoch
    for i, (inputs, labels) in enumerate(progress_bar):

        # move inputs and labels to the target device
        inputs, labels = inputs.to(device), labels.to(device)

        ### TODO - Your code here
        # Clear the gradients
        optimizer.zero_grad()
        # Get the model output
        outputs = model(inputs)
        # Compute loss
        loss = criterion(outputs, labels)
        # Compute gradients
        loss.backward()
        # Update the model parameter
        optimizer.step()
        
        # Accumulate the loss value
        running_loss += loss.item()   ### TODO
        # Get the predicted classes
        _, predicted = outputs.max(1)    ### TODO

        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress_bar.set_postfix({"loss": running_loss / (i + 1), "acc": 100. * correct / total})

    train_loss = running_loss / len(trainloader)
    train_acc = 100. * correct / total
    return train_loss, train_acc


################################################################################
# Define a validation function
################################################################################
def validate(model, valloader, criterion, device):
    """Validate the model"""
    model.eval() # Set to evaluation
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad(): # No need to track gradients
        
        # Put the valloader iterator in tqdm to print progress
        progress_bar = tqdm(valloader, desc="[Validate]", leave=False)

        # Iterate throught the validation set
        for i, (inputs, labels) in enumerate(progress_bar):
            
            # move inputs and labels to the target device
            inputs, labels = inputs.to(device), labels.to(device)
    
            outputs = model(inputs) ### TODO -- inference
            loss = criterion(outputs, labels)    ### TODO -- loss calculation

            running_loss += loss.item()  ### SOLUTION -- add loss from this sample
            _, predicted = torch.max(outputs, 1)   ### SOLUTION -- predict the class

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress_bar.set_postfix({"loss": running_loss / (i+1), "acc": 100. * correct / total})

    val_loss = running_loss/len(valloader)
    val_acc = 100. * correct / total
    return val_loss, val_acc


def main():

    ############################################################################
    #    Configuration Dictionary (Modify as needed)
    ############################################################################
    # It's convenient to put all the configuration in a dictionary so that we have
    # one place to change the configuration.
    # It's also convenient to pass to our experiment tracking tool.


    CONFIG = {
        "model": "SimpleCNN",   # Change name when using a different model
        "batch_size": 128, # run batch size finder to find optimal batch size
        "learning_rate": 0.01,
        "epochs": 5,  # Train for longer in a real scenario
        "num_workers": 4, # Adjust based on your system
        "device": "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",  # Make sure this directory exists
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "seed": 42,
    }

    import pprint
    print("\nCONFIG Dictionary:")
    pprint.pprint(CONFIG)

    ############################################################################
    #      Data Transformation (Example - You might want to modify) 
    ############################################################################

    transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Example normalization
    ])

    ###############
    # TODO Add validation and test transforms - NO augmentation for validation/test
    ###############

    # Validation and test transforms (NO augmentation)
    # AI: Standardized parameters for CIFAR-100
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
    ])   ### TODO -- BEGIN SOLUTION

    ############################################################################
    #       Data Loading
    ############################################################################

    trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                            download=True, transform=transform_train)

    # Split train into train and validation (80/20 split)
    train_size = int(0.8 * len(trainset))   ### TODO -- Calculate training set size
    val_size = len(trainset) - train_size     ### TODO -- Calculate validation set size
    trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])  ### TODO -- split into training and validation sets

    ### TODO -- define loaders and test set
    # Create Dataloader for training data
    trainloader = torch.utils.data.DataLoader(trainset, batch_size = CONFIG["batch_size"], shuffle = True, num_workers = CONFIG["num_workers"])
    # Create DataLoader for validation data
    valloader = torch.utils.data.DataLoader(valset, batch_size = CONFIG["batch_size"], shuffle = False, num_workers = CONFIG["num_workers"])

    # ... (Create validation and test loaders)
    testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size = CONFIG["batch_size"], shuffle = False, num_workers = CONFIG["num_workers"])
    
    ############################################################################
    #   Instantiate model and move to target device
    ############################################################################
    model = SimpleCNN()   # instantiate your model ### TODO
    model = model.to(CONFIG["device"])   # move it to target device

    print("\nModel summary:")
    print(f"{model}\n")

    # The following code you can run once to find the batch size that gives you the fastest throughput.
    # You only have to do this once for each machine you use, then you can just
    # set it in CONFIG.
    SEARCH_BATCH_SIZES = False
    if SEARCH_BATCH_SIZES:
        from utils import find_optimal_batch_size
        print("Finding optimal batch size...")
        optimal_batch_size = find_optimal_batch_size(model, trainset, CONFIG["device"], CONFIG["num_workers"])
        CONFIG["batch_size"] = optimal_batch_size
        print(f"Using batch size: {CONFIG['batch_size']}")
    

    ############################################################################
    # Loss Function, Optimizer and optional learning rate scheduler
    ############################################################################
    # Use cross-entropy for classification
    # AI: For CIFAR-100, it's better to choose nn.CrossEntropyLoss
    criterion = nn.CrossEntropyLoss()   ### TODO -- define loss criterion
    # Use SGD with momentum and weight decay
    # AI: For CIFAR-100, it's better to choose SGD
    optimizer = optim.SGD(model.parameters(), lr=CONFIG["learning_rate"], momentum=0.9, weight_decay = 5e-4)   ### TODO -- define optimizer
    # Reduce LR by half every 10 epochs
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.5)  # Add a scheduler   ### TODO -- you can optionally add a LR scheduler


    # Initialize wandb
    wandb.init(project="-sp25-ds542-challenge", config=CONFIG)
    wandb.watch(model)  # watch the model gradients

    ############################################################################
    # --- Training Loop (Example - Students need to complete) ---
    ############################################################################
    best_val_acc = 0.0

    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG["device"])
        scheduler.step()

        # log to WandB
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"] # Log learning rate
        })

        # Save the best model (based on validation accuracy)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            wandb.save("best_model.pth") # Save to wandb as well

    wandb.finish()

    ############################################################################
    # Evaluation -- shouldn't have to change the following code
    ############################################################################
    import eval_cifar100
    import eval_ood

    # --- Evaluation on Clean CIFAR-100 Test Set ---
    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy: {clean_accuracy:.2f}%")

    # --- Evaluation on OOD ---
    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)

    # --- Create Submission File (OOD) ---
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part1.csv", index=False)
    print("submission_ood_part1.csv created successfully.")

if __name__ == '__main__':
    main()


CONFIG Dictionary:
{'batch_size': 128,
 'data_dir': './data',
 'device': 'cuda',
 'epochs': 5,
 'learning_rate': 0.01,
 'model': 'SimpleCNN',
 'num_workers': 4,
 'ood_dir': './data/ood-test',
 'seed': 42,
 'wandb_project': 'sp25-ds542-challenge'}

Model summary:
SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 50, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=800, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=100, bias=True)
)



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shiychen (shiychen-boston-university). Use `wandb login --relogin` to force relogin


Epoch 1/5 [Train]:   0%|          | 0/313 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch 2/5 [Train]:   0%|          | 0/313 [00:00<?, ?it/s]

...

Evaluating distortion18 (Severity 4):   0%|          | 0/79 [00:00<?, ?it/s]

Exception ignored in: Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x14dd1a639510>Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x14dd1a639510><function _MultiProcessingDataLoaderIter.__del__ at 0x14dd1a639510>
<function _MultiProcessingDataLoaderIter.__del__ at 0x14dd1a639510>

Traceback (most recent call last):

Traceback (most recent call last):
  File "/usr4/dl4ds/shiychen/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr4/dl4ds/shiychen/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
  File "/usr4/dl4ds/shiychen/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
  File "/usr4/dl4ds/shiychen/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
        self._shutdo

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/79 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part1.csv created successfully.
